In [1]:
from civirank import analyzers, parsers, rankers

import json
import numpy as np

import time
import pickle
import pandas as pd

import time
import pandas as pd
import torch
import gc

import warnings
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

/home/joaopn/software/miniconda3/envs/bench/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load sample data
fname = "twitter_test.json"
with open(fname, "r") as fin:
    sample_data_twitter = json.load(fin)
fname = "reddit_test.json"
with open(fname, "r") as fin:
    sample_data_reddit = json.load(fin)
fname = "facebook_test.json"
with open(fname, "r") as fin:
    sample_data_facebook = json.load(fin)

datasets = [sample_data_twitter, sample_data_reddit, sample_data_facebook]

In [5]:
results = {}
batch_sizes = [1,2,4,8,16,32,64]
lim = 100

results = {}
results["twitter"] = {}
results["reddit"] = {}
results["facebook"] = {}

for batch_size in batch_sizes:
    v = rankers.LocalRanker(batch_size=batch_size, lim=lim)

    for dataset in datasets:
        platform = dataset["session"]["platform"]

        tic = time.time()
        v2 = v.rank(dataset)
        toc = time.time()

        results[platform]["batch_" + str(batch_size)] = toc - tic
        
    # Clean up
    del v
    torch.cuda.empty_cache()
    gc.collect()

        

# Puts results into a dataframe
df = pd.DataFrame(results)
df


/home/joaopn/software/miniconda3/envs/bench/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-06-15 06:08:20.006186966 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-06-15 06:08:20.006218266 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
/home/joaopn/software/miniconda3/envs/bench/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downlo

,twitter,reddit,facebook
batch_1,0.450196,0.451281,0.557860
batch_2,0.325783,0.315160,0.357765
batch_4,0.240142,0.196294,0.224578
batch_8,0.196797,0.148977,0.135445
batch_16,0.194891,0.211166,0.104532
batch_32,0.195280,0.296165,0.108724
batch_64,0.205685,0.304029,0.110777
